In [6]:
import numpy as np
import tensorflow as tf
from functools import reduce
import matplotlib.pyplot as plt
import random

gloabal_seed = round(random.random()*100000)

In [7]:
np.random.seed(gloabal_seed)
tf.set_random_seed(gloabal_seed)
sess = tf.InteractiveSession()
num_features = 3
num_samples = 5
state_size = 4
num_of_operations = 3
batch_size = num_samples
max_num_features = 10
datatype = tf.float64


#model placeholders
batchX_placeholder = tf.placeholder(datatype, [batch_size, None], name="batchX")
batchY_placeholder = tf.placeholder(datatype, [batch_size, None], name="batchY")

def add(vec):
    return reduce((lambda x, y: x + y),vec)

def samples_generator(fn, shape, rng, seed=None):
    '''
    Generate random samples for the model:
    @fn - function to be applied on the input features to get the ouput
    @shape - shape of the features matrix (num_samples, num_features)
    @rng - range of the input features to be generated within (a,b)
    @seed  - generation seed
    Outputs a tuple of input and output features matrix
    '''
    x = (rng[1] - rng[0]) * np.random.random_sample(shape) + rng[0]
    y = np.apply_along_axis(fn, 1, x).reshape((shape[0],-1))
    z = np.zeros((shape[0],shape[1] - y.shape[1]))
    y = np.concatenate((y, z), axis=1)
    
    return x,y


_x,_y = samples_generator(add, (num_samples, num_features) , (0, 100))

def tf_multiply(inpt):
    return tf.reshape( tf.reduce_prod(inpt, axis = 1), [batch_size, -1])

def tf_add(inpt):
    return  tf.reshape( tf.reduce_sum(inpt, axis = 1), [batch_size, -1])

def tf_stall(a):
    return a

W_te = tf.Variable(tf.truncated_normal([state_size, num_of_operations], -1*5, 5, dtype=datatype),dtype=datatype, name="W2")

x = tf.Variable(_x, dtype=datatype)
y = tf.Variable(_y, dtype=datatype)
W_te.initializer.run()
x.initializer.run()
y.initializer.run()

gloabal_seed

64550

In [8]:
_x

array([[ 84.19388261,  86.86347389,  23.75188749],
       [ 13.19355386,  69.44692959,  92.72612234],
       [ 97.77557888,  96.82252771,   3.65927751],
       [ 88.81764449,  52.37139201,  96.97827122],
       [ 30.65724851,  53.9719062 ,  30.12296349]])

In [9]:
W_te.name

'W2_1:0'

In [24]:
add    = tf_add(x)
mult   = tf_multiply(x)
stall  = tf_stall(x)
values = tf.concat([add, mult, stall], 1)
#print(values.eval())
#padding = tf.constant()
print(add.eval())
print(mult.eval())
print(stall.eval())

[[ 194.809244  ]
 [ 175.3666058 ]
 [ 198.25738409]
 [ 238.16730773]
 [ 114.7521182 ]]
[[ 173706.41561076]
 [  84960.47708269]
 [  34641.93627879]
 [ 451094.78522039]
 [  49842.36332631]]
[[ 84.19388261  86.86347389  23.75188749]
 [ 13.19355386  69.44692959  92.72612234]
 [ 97.77557888  96.82252771   3.65927751]
 [ 88.81764449  52.37139201  96.97827122]
 [ 30.65724851  53.9719062   30.12296349]]


In [21]:
init_state = tf.Variable(np.zeros([batch_size, state_size]), dtype=datatype)
argmax_dum = tf.Variable([2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  dtype=datatype)
W = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=datatype)
b = tf.Variable(np.zeros((state_size)), dtype=datatype)

W2 = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=datatype)
b2 = tf.Variable(np.zeros((num_of_operations)), dtype=datatype)

dummy_matrix = tf.zeros([batch_size, num_features], datatype, name="dummy_constant")

init_state.initializer.run()
argmax_dum.initializer.run()
W.initializer.run()
b.initializer.run()
W2.initializer.run()
b2.initializer.run()
#dummy_matrix.initializer.run()

current_state = init_state
current_input = x
      
input_and_state_concatenated = tf.concat([current_input, current_state], 1)  # Increasing number of columns
next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
current_state = next_state
    
    #calculate softmax and produce the mask of operations
logits  = tf.matmul(next_state, W2) + b2 #Broadcasted addition
softmax = tf.nn.softmax(logits)
softmax_10000 = tf.nn.softmax(logits)
softplus = tf.nn.softplus(logits)
softsign = tf.nn.softsign(logits)
argmax  = tf.argmax(softmax, 1)

#lengths_transposed = tf.expand_dims(argmax, 0)
#onehot  = tf.one_hot(argmax, num_of_operations)
stall_width = tf.shape(stall)[1] - 1 
stall_select = tf.slice(softmax, [0,2], [batch_size,1])
multiples = [1, stall_width]
stall_mask = tf.tile(stall_select, multiples)
mask = tf.concat([softmax, stall_mask], 1)
#mask = tf.cast(mask, tf.bool)
#apply mask
#output = tf.boolean_mask(values,mask)
output = values * mask
nn_soft = tf.nn.softmax(logits)
output_width = tf.shape(output)[1]
#zeros_slice = tf.slice(dummy_matrix, [0,0], [batch_size, max_num_features - output_width])
#dum_plus_input = tf.concat([output, zeros_slice], 1)
#print(dum_plus_input.eval())

'''
print(x.eval())
print("stall_select")
print(stall_select.eval())
print("mask")
print(mask.eval())
print("logits")
print(logits.eval())
print("softmax")
print(softmax.eval())
print("softplus")
print(softplus.eval())
print("softsign")
print(softsign.eval())
print("softmax*10000")
print(softmax_10000.eval())
print("onehot")
print(onehot.eval())
'''
#print(mask.eval())
#print(mask.eval())
#print(output.eval())
print(softmax.eval())

[[ 0.14723534  0.54142297  0.31134169]
 [ 0.14723534  0.54142297  0.31134169]
 [ 0.14723534  0.54142297  0.31134169]
 [ 0.14723534  0.54142297  0.31134169]
 [ 0.14723534  0.54142297  0.31134169]]


In [26]:
add_softmax   = tf.slice(softmax, [0,0], [batch_size,1])
mult_softmax  = tf.slice(softmax, [0,1], [batch_size,1])
stall_softmax = tf.slice(softmax, [0,2], [batch_size,1])

print(add_softmax.eval())
print(mult_softmax.eval())
print(stall_softmax.eval())

add_width   = tf.shape(add)[1]
mult_width  = tf.shape(mult)[1]
stall_width = tf.shape(stall)[1]

padd_Add = tf.constant(np[[0,0],[0,num_features - add_width]])
add_final = tf.pad(add, padd_Add, "CONSTANT") 
mult_final = tf.pad(mult, [[0,0],[0,num_features - mult_width]], "CONSTANT") 
stall_final = tf.pad(stall, [[0,0],[0,num_features - stall_width]], "CONSTANT")

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

#tf.multiply()
add_final   = tf.multiply(add_final, add_softmax)
mult_final  = tf.multiply(mult_final, mult_softmax)
stall_final = tf.multiply(stall_final, stall_softmax)

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

[[ 0.14723534]
 [ 0.14723534]
 [ 0.14723534]
 [ 0.14723534]
 [ 0.14723534]]
[[ 0.54142297]
 [ 0.54142297]
 [ 0.54142297]
 [ 0.54142297]
 [ 0.54142297]]
[[ 0.31134169]
 [ 0.31134169]
 [ 0.31134169]
 [ 0.31134169]
 [ 0.31134169]]


TypeError: List of Tensors when single Tensor expected

In [17]:
add_width   = tf.shape(add)[1]
mult_width  = tf.shape(mult)[1]
stall_width = tf.shape(stall)[1]


add_final   = add * add_softmax
mult_final  = mult * mult_softmax
stall_final = stall * stall_softmax

##conact add and mult results with zeros matrix
add_final = tf.concat([add_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - add_width])], 1) 
mult_final = tf.concat([mult_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - mult_width])], 1) 

output_final = add_final + mult_final + stall_final 

print(add_width.eval())
print(mult_width.eval())
print(stall_width.eval())


print(add.eval())
print(mult.eval())
print(stall.eval())

print(add_softmax.eval())
print(mult_softmax.eval())
print(stall_softmax.eval())

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

print(output_final.eval())

1
1
3
[[ 194.809244  ]
 [ 175.3666058 ]
 [ 198.25738409]
 [ 238.16730773]
 [ 114.7521182 ]]
[[ 173706.41561076]
 [  84960.47708269]
 [  34641.93627879]
 [ 451094.78522039]
 [  49842.36332631]]
[[ 84.19388261  86.86347389  23.75188749]
 [ 13.19355386  69.44692959  92.72612234]
 [ 97.77557888  96.82252771   3.65927751]
 [ 88.81764449  52.37139201  96.97827122]
 [ 30.65724851  53.9719062   30.12296349]]
[[ 0.11956474]
 [ 0.11956474]
 [ 0.11956474]
 [ 0.11956474]
 [ 0.11956474]]
[[ 0.55501353]
 [ 0.55501353]
 [ 0.55501353]
 [ 0.55501353]
 [ 0.55501353]]
[[ 0.32542173]
 [ 0.32542173]
 [ 0.32542173]
 [ 0.32542173]
 [ 0.32542173]]
[[ 23.29231618   0.           0.        ]
 [ 20.96766224   0.           0.        ]
 [ 23.70459214   0.           0.        ]
 [ 28.4764117    0.           0.        ]
 [ 13.72030692   0.           0.        ]]
[[  96409.41101023       0.               0.        ]
 [  47154.2143443        0.               0.        ]
 [  19226.74335976       0.               0.     

In [28]:

def custom_softmax(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = tf.log(sum_exponents)
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( inpt / soft_max, [batch_size, -1])

def custom_softmax2(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = sum_exponents
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( tf.exp(inpt) / soft_max, [batch_size, -1])
x_cust = custom_softmax(x)
x_cust2 = custom_softmax2(x)
print(x.eval())
#print(x_cust2.eval())
print(tf.nn.softmax(x*10000).eval())

[[ 88.83499413]
 [ 85.56388494]
 [ 19.11761486]
 [ 92.13705181]
 [ 82.50776821]]
[[  3.80669209e+38]
 [  1.44518379e+37]
 [  2.00758824e+08]
 [  1.03422059e+40]
 [  6.80249839e+35]]
[[ 28.43746689  88.83499413]
 [ 44.79539098  85.56388494]
 [ 19.11761484   1.44401848]
 [ 92.13705181  40.86930991]
 [  2.55786472  82.50776821]]
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]


In [29]:
math_error = tf.multiply(tf.constant(0.5, dtype=datatype), tf.square(tf.subtract(tf.reduce_sum(output, axis = 1, name="red_output") , batchY_placeholder, name="sub_otput_batchY"), name="squar_error"), name="mult_with_0.5")

total_loss = tf.reduce_sum(math_error, name="red_total_loss")

grads = tf.gradients(total_loss, [W,b,W2,b2], name="comp_gradients")
train_step = tf.train.AdamOptimizer(0.1, name="AdamOpt").minimize(total_loss, name="min_loss")

In [30]:
def calc_loss(output):
    #reduced_output = tf.reshape( tf.reduce_sum(output, axis = 1, name="red_output"), [batch_size, -1])
    math_error = tf.multiply(tf.constant(0.5, dtype=datatype), tf.square(tf.subtract(output , y, name="sub_otput_batchY"), name="squar_error"), name="mult_with_0.5")
    
                             
    total_loss = tf.reduce_sum(math_error, name="red_total_loss")
    return total_loss, math_error


loss, mathh = calc_loss(output_final)

In [31]:
mathh.eval()

array([[  1.30718378e+06,   1.69663271e+02],
       [  3.11424301e+06,   1.57398529e+02],
       [  1.00464276e+01,   4.48384279e-02],
       [  3.01961583e+06,   3.59099076e+01],
       [  2.33981706e+03,   1.46356992e+02]])

In [198]:
prng = np.random.RandomState(1234)

In [115]:
sess.close()

In [199]:
prng.random_sample((10,5))

array([[ 0.19151945,  0.62210877,  0.43772774,  0.78535858,  0.77997581],
       [ 0.27259261,  0.27646426,  0.80187218,  0.95813935,  0.87593263],
       [ 0.35781727,  0.50099513,  0.68346294,  0.71270203,  0.37025075],
       [ 0.56119619,  0.50308317,  0.01376845,  0.77282662,  0.88264119],
       [ 0.36488598,  0.61539618,  0.07538124,  0.36882401,  0.9331401 ],
       [ 0.65137814,  0.39720258,  0.78873014,  0.31683612,  0.56809865],
       [ 0.86912739,  0.43617342,  0.80214764,  0.14376682,  0.70426097],
       [ 0.70458131,  0.21879211,  0.92486763,  0.44214076,  0.90931596],
       [ 0.05980922,  0.18428708,  0.04735528,  0.67488094,  0.59462478],
       [ 0.53331016,  0.04332406,  0.56143308,  0.32966845,  0.50296683]])

In [208]:
_x[0:2]

array([[ 21.88666268,  34.73962374],
       [ 40.79798811,  43.39526771]])

In [ ]:
q = tf.FIFOQueue(capacity=2, dtypes=datatype)
q.enqueue(x)
q.dequeue().eval()

In [ ]:
for batch_idx in range(num_batches):
            start_idx = batch_size * batch_idx
            end_idx   = batch_size * batch_idx + batch_idx

            batchX = x[start_idx:end_idx]
            batchY = y[start_idx:end_idx]
            